-제시된 spotify_top_songs_audio_features.csv 파일을 이용하여 streams를 예측하는 모델을 완성하고

완성된 모델의 개발과정에서 작성한 코드와 테스트 데이터에 대한 결정계수를 답안으로 작성하시오

 

In [148]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt

import statsmodels.api as sm 

In [2]:
from matplotlib import font_manager, rc
import matplotlib
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family=font_name)

In [217]:
df = pd.read_csv("./data/spotify_top_songs_audio_features.csv")
df.head()

,id,artist_names,track_name,source,key,mode,time_signature,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,loudness,tempo,duration_ms,weeks_on_chart,streams
0,000xQL6tZNLJzIrtIgxqSl,"ZAYN, PARTYNEXTDOOR",Still Got Time (feat. PARTYNEXTDOOR),RCA Records Label,G,Major,4 beats,0.748,0.627,0.0639,0.13100,0.0,0.0852,0.524,-6.029,120.963,188491,17,107527761
1,003eoIwxETJujVWmNFMoZy,Alessia Cara,Growing Pains,Def Jam Recordings,C#/Db,Minor,4 beats,0.353,0.755,0.7330,0.08220,0.0,0.3900,0.437,-6.276,191.153,193680,2,9944865
2,003vvx7Niy0yvhvHt4a68B,The Killers,Mr. Brightside,Island Records,C#/Db,Major,4 beats,0.352,0.911,0.0747,0.00121,0.0,0.0995,0.236,-5.230,148.033,222973,125,512388123
3,00B7TZ0Xawar6NZ00JFomN,"Cardi B, Chance the Rapper",Best Life (feat. Chance The Rapper),Atlantic/KSR,A,Major,4 beats,0.620,0.625,0.5530,0.28700,0.0,0.3140,0.665,-7.438,167.911,284856,2,11985346
4,00Blm7zeNqgYLPtW6zg8cj,"Post Malone, The Weeknd",One Right Now (with The Weeknd),Republic Records,C#/Db,Major,4 beats,0.687,0.781,0.0530,0.03610,0.0,0.0755,0.688,-4.806,97.014,193507,30,301860377


In [218]:
df.dtypes

id                   object
artist_names         object
track_name           object
source               object
key                  object
mode                 object
time_signature       object
danceability        float64
energy              float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
loudness            float64
tempo               float64
duration_ms           int64
weeks_on_chart        int64
streams               int64
dtype: object

In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6513 entries, 0 to 6512
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                6513 non-null   object 
 1   artist_names      6513 non-null   object 
 2   track_name        6513 non-null   object 
 3   source            6513 non-null   object 
 4   key               6513 non-null   object 
 5   mode              6513 non-null   object 
 6   time_signature    6513 non-null   object 
 7   danceability      6513 non-null   float64
 8   energy            6513 non-null   float64
 9   speechiness       6513 non-null   float64
 10  acousticness      6513 non-null   float64
 11  instrumentalness  6513 non-null   float64
 12  liveness          6513 non-null   float64
 13  valence           6513 non-null   float64
 14  loudness          6513 non-null   float64
 15  tempo             6513 non-null   float64
 16  duration_ms       6513 non-null   int64  


In [220]:
df.shape

(6513, 19)

In [221]:
df.describe()

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,loudness,tempo,duration_ms,weeks_on_chart,streams
count,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6.513000e+03
mean,0.681731,0.636522,0.121933,0.236761,0.012469,0.180168,0.492412,-6.350667,122.117244,202566.684170,13.614924,1.089647e+08
std,0.141787,0.164813,0.113441,0.244784,0.075151,0.138054,0.227001,2.536114,29.416097,49199.592333,24.084051,2.276695e+08
min,0.150000,0.021800,0.023200,0.000008,0.000000,0.019700,0.032000,-34.475000,46.718000,30133.000000,1.000000,2.525159e+06
25%,0.591000,0.534000,0.044000,0.044400,0.000000,0.097400,0.316000,-7.564000,98.007000,173038.000000,1.000000,8.695060e+06
50%,0.698000,0.651000,0.072200,0.145000,0.000000,0.124000,0.489000,-5.983000,120.034000,198367.000000,4.000000,2.712856e+07
75%,0.785000,0.759000,0.163000,0.356000,0.000041,0.219000,0.669000,-4.673000,142.025000,226003.000000,16.000000,9.951456e+07
max,0.985000,0.989000,0.966000,0.994000,0.953000,0.977000,0.982000,1.509000,212.117000,690732.000000,367.000000,3.528450e+09


In [222]:
df.isna().sum()

id                  0
artist_names        0
track_name          0
source              0
key                 0
mode                0
time_signature      0
danceability        0
energy              0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
loudness            0
tempo               0
duration_ms         0
weeks_on_chart      0
streams             0
dtype: int64

In [223]:
df.columns

Index(['id', 'artist_names', 'track_name', 'source', 'key', 'mode',
       'time_signature', 'danceability', 'energy', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'loudness',
       'tempo', 'duration_ms', 'weeks_on_chart', 'streams'],
      dtype='object')

In [224]:
df["artist_names"].value_counts()

artist_names
Taylor Swift                     175
Drake                            111
BTS                               71
Bad Bunny                         70
Ariana Grande                     60
                                ... 
Becky G, NATTI NATASHA             1
Shakira, Manuel Turizo             1
Martin Garrix, Bono, The Edge      1
Lauv, Julia Michaels               1
Kendrick Lamar, Tanna Leone        1
Name: count, Length: 3001, dtype: int64

In [225]:
rdf = df[[ 'energy','tempo', 'duration_ms', 'weeks_on_chart', 'streams']]

In [226]:
rdf_columns = [ 'energy','tempo', 'duration_ms', 'weeks_on_chart', 'streams']

In [227]:
rdf.head()

,energy,tempo,duration_ms,weeks_on_chart,streams
0,0.627,120.963,188491,17,107527761
1,0.755,191.153,193680,2,9944865
2,0.911,148.033,222973,125,512388123
3,0.625,167.911,284856,2,11985346
4,0.781,97.014,193507,30,301860377


In [228]:
rdf.corr()

,energy,tempo,duration_ms,weeks_on_chart,streams
energy,1.000000,0.095561,0.013937,0.009613,0.001399
tempo,0.095561,1.000000,-0.017492,-0.017994,-0.012052
duration_ms,0.013937,-0.017492,1.000000,0.031530,0.014761
weeks_on_chart,0.009613,-0.017994,0.031530,1.000000,0.855880
streams,0.001399,-0.012052,0.014761,0.855880,1.000000


In [229]:
rdf.describe()

,energy,tempo,duration_ms,weeks_on_chart,streams
count,6513.000000,6513.000000,6513.000000,6513.000000,6.513000e+03
mean,0.636522,122.117244,202566.684170,13.614924,1.089647e+08
std,0.164813,29.416097,49199.592333,24.084051,2.276695e+08
min,0.021800,46.718000,30133.000000,1.000000,2.525159e+06
25%,0.534000,98.007000,173038.000000,1.000000,8.695060e+06
50%,0.651000,120.034000,198367.000000,4.000000,2.712856e+07
75%,0.759000,142.025000,226003.000000,16.000000,9.951456e+07
max,0.989000,212.117000,690732.000000,367.000000,3.528450e+09


In [230]:
x = rdf.drop('streams', axis = 1)
y = rdf["streams"]

In [231]:
y

0       107527761
1         9944865
2       512388123
3        11985346
4       301860377
          ...    
6508     17583532
6509     10526635
6510      5404939
6511     12872010
6512      6568514
Name: streams, Length: 6513, dtype: int64

In [232]:
ss = StandardScaler()

x = ss.fit_transform(x)

In [233]:
y.value_counts()

streams
107527761    1
50675911     1
169278501    1
326194123    1
378241571    1
            ..
80000703     1
201317070    1
45071942     1
5679681      1
6568514      1
Name: count, Length: 6513, dtype: int64

In [234]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 8)

In [235]:
x_train.shape, x_test.shape

((5210, 4), (1303, 4))

In [236]:
len(x_train), len(y_test)

(5210, 1303)

In [237]:
lr = LinearRegression()

In [238]:
lr.fit(x_train, y_train)

LinearRegression()

In [239]:
r_square = lr.score(x_test, y_test)
r_square

0.7925508351367233

In [240]:
lr.coef_

array([-2.27309185e+06,  1.18585551e+06, -2.31197871e+06,  1.96729492e+08])

In [241]:
lr.intercept_

109079927.93622471

In [242]:
y_pred = lr.predict(x_test)

In [243]:
mse = mean_squared_error(y_test, y_pred)
mse

1.3135373721267508e+16

In [244]:
# rmse = mean_squared_error(y_test, y_pred, squared = False)
# rmse

In [245]:
y_test

1546      6127103
1963     35262377
5076      4426272
5904      6342388
5459     39022272
          ...    
2485      7273393
169     316957596
5643     27977667
2499     30041039
6415     20263638
Name: streams, Length: 1303, dtype: int64

In [246]:
y_pred

array([3.27370202e+08, 3.18386234e+07, 2.60047455e+06, ...,
       4.68276388e+07, 4.09998022e+07, 1.73889824e+07])

In [215]:
# x_train, x_test 상수항 추가(add_constant)
x_train_new = sm.add_constant(x_train)
x_test_new = sm.add_constant(x_test)

# full_mod ( 다 넣었다) / OLS : 선형회귀(최소제곱법)
full_mod = sm.OLS(y_train, x_train_new)

In [216]:
x_train.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'